In [1]:
import os, sys

recsys_cf = os.environ.get("RECSYS_STORY")
sys.path.append(recsys_cf)
os.chdir(recsys_cf)

from lib.tools import *

In [2]:
from urllib.parse import urlparse

###### spark 

In [4]:
sp = spark(**SPARK_CONFIG)
hive = sp.sql

###### load data 

In [4]:
visit = load_table(SBX_TEAM_DIGITCAMP, VISIT, hive).cache()

In [5]:
visit.show(3)

,visitNumber,cid,commonSegmentoUID,sbbolUserId,hitPagePath,hitPageTitle,sessionStartTime,sessionDate,hitType,hitNumber,hitTime,eventCategory,eventAction,eventLabel,hitPageHostName,ctl_loading,sessionId,deviceIsMobile,deviceMobileDeviceModel,deviceDeviceCategory,geoRegion,geoCity
0,2,1304721468.1624081086,48NmCasKmfcx,None,/ru/s_m_business/credits/warranty,Банковская гарантия от Сбербанка — СберБанк,1624756863,2021-06-27,PAGE,2,1224526,None,None,None,www.sberbank.ru,9663295,1624756863,True,SM-A013F,mobile,Primorsky Krai,Vladivostok
1,70,1771292124.1608378202,None,None,/journal/article/2360,Активный возраст,1624780273,2021-06-27,PAGE,2,1,None,None,None,sberbankaktivno.ru,9663295,1624780273,True,SM-J400F,mobile,Saint Petersburg,Saint Petersburg
2,1,999719353.1624750791,None,None,/how_it_works,Все о программе лояльности СберСпасибо,1624750793,2021-06-27,PAGE,2,19564,None,None,None,spasibosberbank.ru,9663295,1624750793,True,COL-L29,mobile,Buryatia,Ulan-Ude


In [214]:
begin_date = '2021-06-14'
end_date = '2021-06-20'

In [216]:
visit = hive.sql("""
 select cid,
        sbbolUserId,
        hitPagePath,
        hitPageTitle,
        sessionId,
        sessionDate,
        sessionStartTime,
        hitTime,
        hitType,
        eventCategory,
        eventAction,
        eventLabel,
        hitPageHostName
        
 from cap_external_google_dda_external_google_analytics.visit
where 1 = 1
  and sessiondate >= "{}"
  and sessiondate <= "{}"
  and (hitPageHostName = 'sbi.sberbank.ru'
       OR hitPagePath like '%/s_m_business%')
""".format(begin_date, end_date)).cache()

### Препроцессинг 

отчистка путей от мусора

In [217]:
@udf(stypes.StringType())
def get_url_path(url):
        try:
            url_parse = urlparse(url)
        except Exception as e:
            print(e)
            return None

        return url_parse.path

In [218]:
visit_cols = visit.withColumn("hitPagePath", get_url_path(f.col("hitPagePath")))\
                  .filter(f.col("hitPagePath") != "/" )

In [219]:
visit_ts = visit_cols.withColumn("timestamp", 
                                 (f.col("sessionStartTime") + f.col("hitTime") / 1000).cast(stypes.LongType())) \
                     .drop("sessionStartTime", "hitTime")

In [220]:
visit_ts.cache()

DataFrame[cid: string, sbbolUserId: string, hitPagePath: string, hitPageTitle: string, sessionId: bigint, sessionDate: string, hitType: string, eventCategory: string, eventAction: string, eventLabel: string, hitPageHostName: string, timestamp: bigint]

In [221]:
visit_ts.count()

269686079

### Все клики по сторям в СББОЛе 

Для начала соберем датасет, состоящий из целевых событий юзеров

In [222]:
visit_sbbol = visit_ts.filter("hitPageHostName = 'sbi.sberbank.ru'")\
                      .select("sbbolUserId",
                              "hitPagePath",
                              "hitPageTitle",
                              "sessionId",
                              "timestamp",
                              "sessionDate",
                              "hitType",
                              "eventCategory",
                              "eventAction",
                              "eventLabel",
                              "hitPageHostName",
                              "sessionId")

In [223]:
visit_sbbol.cache()

DataFrame[sbbolUserId: string, hitPagePath: string, hitPageTitle: string, sessionId: bigint, timestamp: bigint, sessionDate: string, hitType: string, eventCategory: string, eventAction: string, eventLabel: string, hitPageHostName: string, sessionId: bigint]

In [224]:
visit_sbbol.filter("hitPagePath = '/main'").select("sbbolUserId").distinct().count()

746741

Выделим клики по сторям

In [225]:
STORY_CLICK = '''hitPagePath = '/main' AND hitType = 'EVENT'
                 AND
                 eventCategory = '[operations]: offers' AND eventAction  = 'open card'
'''

In [226]:
sbbol_story = visit_sbbol.filter(STORY_CLICK)

Выделеним продукт из клика по сторе

In [227]:
pattern1 = 'product\: [^]]*]{1}'
story_product = sbbol_story.withColumn('product',f.regexp_extract("eventLabel", pattern1, 0))\
                           .withColumn("product",
                                       f.col("product").substr(f.lit(10), f.length("product") - 10))


In [ ]:
story_product.show(1)

In [229]:
sbbol_story_click_sdf = story_product.select('sbbolUserId',
                                             'hitPagePath',
                                             'timestamp',
                                             'sessionDate',
                                             'hitType',
                                             'eventCategory',
                                             'eventAction',
                                             'product')

In [ ]:
sbbol_story_click_sdf.show(3)

###### Статистика

In [231]:
stories_user_cnt = sbbol_story_click_sdf.select("sbbolUserId").distinct().count()
all_users_sbbol = visit_sbbol.filter("hitPagePath = '/main'").select("sbbolUserId").distinct().count()
all_users_prod_sbbol = sbbol_story_click_sdf.filter("product <> 'none'")\
                                           .select("sbbolUserId").distinct().count()



all_clicks = sbbol_story_click_sdf.count()
all_clicks_prod = sbbol_story_click_sdf.filter("product <> 'none'").count()



all_prod_cnt = sbbol_story_click_sdf.select("product").distinct().count()

In [232]:
print("Всего юзеров главнй страницы СББОЛа:", all_users_sbbol)
print("Юзеров с кликами по сторям в СББОЛе:", stories_user_cnt)
print("Юзеров с кликами по продуктовым сторям в СББОЛе:", all_users_prod_sbbol)
print("Процент юзеров с кликами по сторям:", round(stories_user_cnt/all_users_sbbol * 100, 2), '%')
print("Процент юзеров с кликами по продуктовым сторям:", round(all_users_prod_sbbol/all_users_sbbol * 100, 2), '%')

print("\n \nClicks:")
print("Кликов по сторям всего:", all_clicks)
print("Кликов по продуктовым сторям всего:", all_clicks_prod)



print("\nВсего различных продуктов в сторях:", all_prod_cnt)


Всего юзеров главнй страницы СББОЛа: 746741
Юзеров с кликами по сторям в СББОЛе: 16323
Юзеров с кликами по продуктовым сторям в СББОЛе: 4278
Процент юзеров с кликами по сторям: 2.19 %
Процент юзеров с кликами по продуктовым сторям: 0.57 %

 
Clicks:
Кликов по сторям всего: 24646
Кликов по продуктовым сторям всего: 5530

Всего различных продуктов в сторях: 62


Статистика охвата сторей по юзерам

In [233]:
sbbol_story_click_sdf.groupBy("product").agg(f.countDistinct("sbbolUserId").alias("cntDistinctUsers"),
                                             f.count("sbbolUserId").alias("cntClicks"),
                                             (f.countDistinct("sbbolUserId") /
                                              stories_user_cnt *100).alias("cntDistinctUserPrc")
                                            )\
                     .orderBy(f.col("cntDistinctUsers").desc()).show(30)

,product,cntDistinctUsers,cntClicks,cntDistinctUserPrc
0,none,13097,19055,80.236476
1,kredit,849,1014,5.201250
2,lyurist,697,800,4.270048
3,rabota_ru,666,773,4.080132
4,docdoc,495,520,3.032531
5,credit_card,307,361,1.880782
6,korpkarta,181,210,1.108865
7,refundvat,106,114,0.649390
8,e-acquiring,91,106,0.557496
9,leasing,82,96,0.502359


### SM_Business

In [234]:
visit_smb = visit_ts.select("cid",
                            "sessionId",
                            "timestamp",
                            "hitPagePath",
                            "eventAction",
                            
                            "eventLabel",
                            "sessionDate")\
                     .filter("hitPagePath like '%/s_m_business%'")\
                     .cache()

In [235]:
w=Window.partitionBy('cid').orderBy('sessionId', 'timestamp')
 
visit_smb_click = visit_smb.withColumn('next_state_m1', f.lead(f.col('hitPagePath'),count=1).over(w))\
                           .withColumn('next_state_m2', f.lead(f.col('hitPagePath'),count=2).over(w))
     
visit_smb_click = visit_smb_click.withColumn('clickBannerAndGo2Landing',          
                                               ((f.col("hitPagePath") == "/ru/s_m_business")|
                                                (f.col("hitPagePath") == "/ru/s_m_business/main/")|
                                                (f.col("hitPagePath") == "/ru/s_m_business_alt")
                                               ) &
                                               (
                                                f.col('eventAction').like("%click_any%")|
                                                f.col('eventAction').like("%click_banner%")
                                               )&
                                               ((f.regexp_extract(f.col("next_state_m1"),
                                                                           "(?:/ru/s_m_business/)([a-zA-Z_/]+)",1) != ''
                                                )|
                                                (f.regexp_extract(f.col("next_state_m2"),
                                                                           "(?:/ru/s_m_business/)([a-zA-Z_/]+)",1) != ''
                                                )                                               
                                               )  
                    )

In [236]:
banner_click_any_cond = (
                     ((f.col("hitPagePath") == "/ru/s_m_business")|
                      (f.col("hitPagePath") == "/ru/s_m_business/main/")|
                      (f.col("hitPagePath") == "/ru/s_m_business_alt")
                     )&
                     ((f.col('eventAction').like('%click_banner%'))|
                      (f.lower(f.col('eventAction')).like('%click_any%'))
                     )
                    )
banner_click_cond = (
                     ((f.col("hitPagePath") == "/ru/s_m_business")|
                      (f.col("hitPagePath") == "/ru/s_m_business/main/")|
                      (f.col("hitPagePath") == "/ru/s_m_business_alt")
                     )&
                     ((f.col('eventAction').like('%click_banner%'))
                     )
                    )
banner_any_cond = (
                     ((f.col("hitPagePath") == "/ru/s_m_business")|
                      (f.col("hitPagePath") == "/ru/s_m_business/main/")|
                      (f.col("hitPagePath") == "/ru/s_m_business_alt")
                     )&
                     (
                      (f.lower(f.col('eventAction')).like('%click_any%'))
                     )
                    )

smb_page_cond = ((f.col("hitPagePath") == "/ru/s_m_business")|
                                 (f.col("hitPagePath") == "/ru/s_m_business/main/")|
                                                (f.col("hitPagePath") == "/ru/s_m_business_alt"))

In [237]:
visit_smb_click = visit_smb_click.filter("clickBannerAndGo2Landing = 1").cache()


###### Статистика кликов s_m_business

In [238]:
cid_all_cnt = visit_smb.filter(smb_page_cond).select("cid").distinct().count()
cid_click_lend_cnt = visit_smb_click.select('cid').distinct().count()
Ea_click_banner_users_cnt = visit_smb.filter(banner_click_cond).select("cid").distinct().count()






all_clicks_banner = visit_smb_click.count()
Ea_click_banner_cnt = visit_smb.filter(banner_click_cond).count()

In [239]:
print("Всего юзеров главнй страницы s_m_business:", cid_all_cnt)
print("Юзеров с кликами по баннерам и с переходом на лендинг в s_m_business:", cid_click_lend_cnt)
print("Юзеров с ea 'click_banner' по баннерам или с переходом на лендинг в s_m_business:", Ea_click_banner_users_cnt)

print("Процент юзеров с кликами по баннерам или с переходом на лендинг в s_m_business:", round(cid_click_lend_cnt/cid_all_cnt * 100, 2), '%')



print("\n \nClicks:")
print("Кликов по баннерам с переходами на лендинг:", all_clicks_banner)
print("Кликов с ea 'click_banner' по баннерам и с переходом на лендинг в s_m_business:", Ea_click_banner_cnt)

Всего юзеров главнй страницы s_m_business: 107217
Юзеров с кликами по баннерам и с переходом на лендинг в s_m_business: 20535
Юзеров с ea 'click_banner' по баннерам или с переходом на лендинг в s_m_business: 903
Процент юзеров с кликами по баннерам или с переходом на лендинг в s_m_business: 19.15 %

 
Clicks:
Кликов по баннерам с переходами на лендинг: 33182
Кликов с ea 'click_banner' по баннерам и с переходом на лендинг в s_m_business: 999


Сколько событий click_any и click_banner

In [240]:
visit_smb.filter(banner_click_any_cond).count()

318758

In [241]:
visit_smb.filter(banner_click_cond).count()

999

###### на примере баннера по эквайрингу 

Потыкал в эквайринг в баннерной карусели и посмотрел на возможные хиты

In [242]:
equaring_click_cnt = visit_smb.filter(banner_click_cond).filter("eventLabel like '%эква'").count()
near_equaring_click_cnt = visit_smb.filter(banner_any_cond).filter("eventLabel like 'Промотариф на эква%3-n9%'").count()

In [243]:
print("Кликов на кнопку подробнее по эквайренгу в каруселе s_m_business:", equaring_click_cnt)
print("Кликов рядом с областью по эквайрингу в s_m_business:", near_equaring_click_cnt)

Кликов на кнопку подробнее по эквайренгу в каруселе s_m_business: 0
Кликов рядом с областью по эквайрингу в s_m_business: 216


In [244]:
visit_smb.filter(banner_click_cond).show()

,cid,sessionId,timestamp,hitPagePath,eventAction,eventLabel,sessionDate
0,1296166882.1591263875,1624217572,1624217609,/ru/s_m_business,click_banner,2_Переводы физлицам до 300 000 ₽ — бесплатно для ИП,2021-06-20
1,1307621448.1624178558,1624178461,1624178475,/ru/s_m_business,click_banner,1_Участвуйте в проекте СберРядом,2021-06-20
2,791870807.1591277959,1624097385,1624097470,/ru/s_m_business,click_banner,2_Переводы физлицам до 300 000 ₽ — бесплатно для ИП,2021-06-19
3,269217489.1602075502,1624180412,1624180518,/ru/s_m_business,click_banner,1_Участвуйте в проекте СберРядом,2021-06-20
4,116554582.1622060197,1624139757,1624141397,/ru/s_m_business,click_banner,4_Промотариф на эквайринг,2021-06-20
5,1574162090.1620230756,1624109302,1624109308,/ru/s_m_business,click_banner,1_Участвуйте в проекте СберРядом,2021-06-19
6,390000491.1623156761,1624107901,1624108155,/ru/s_m_business,click_banner,3_Онлайн-кредит для бизнеса,2021-06-19
7,244498060.1620665908,1624078579,1624081351,/ru/s_m_business,click_banner,1_Участвуйте в проекте СберРядом,2021-06-19
8,1403937606.1622275797,1624128964,1624131054,/ru/s_m_business,click_banner,1_Участвуйте в проекте СберРядом,2021-06-19
9,1403937606.1622275797,1624128964,1624131088,/ru/s_m_business,click_banner,2_Переводы физлицам до 300 000 ₽ — бесплатно для ИП,2021-06-19


сколько кликов по событиям на странице s_m_business, ведущим на страницу с эквайрингом

In [245]:
ecuaring_df = visit_smb_click.filter('''
                                           next_state_m2 like '%/bankingservice/acquiring_total%' 
                                        OR next_state_m1 like '%/bankingservice/acquiring_total%' 
                                     ''' )

In [246]:
ecuaring_df.count()

1719

In [ ]:
ecuaring_df.show(5)

Посмотрел, как на главной странице выглядит переход по ссылке на эквайринг через гиперссылку

In [248]:
ecuaring_df.filter("eventLabel like 'эквайринг_'").count()

127

In [ ]:
ecuaring_df.filter("eventLabel like 'эквайринг_'").show()